In [15]:
import utils

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Normalization

In [2]:
input_cols = ['year',
              'matches_won_percent',
              'podium_score_yearly',
              'matches_won_percent_2',
              'podium_score_yearly_2',]
output_col = 'winner'

In [3]:
matches = utils.get_matches(with_team_stats=True, duplicate_with_reversed=True)
matches

,id,year,team1,score1,score2,team2,winner,matches_played,matches_won,years_played,...,podium_score_yearly,cups_won_yearly,matches_played_2,matches_won_2,years_played_2,podium_score_2,cups_won_2,matches_won_percent_2,podium_score_yearly_2,cups_won_yearly_2
0,0,1950,Brazil,4,0,Mexico,True,77.0,65.0,17.0,...,6.000000,0.294118,35.0,14.0,14.0,0.0,0.0,0.400000,0.000000,0.0
1,1,1950,Yugoslavia,3,0,Switzerland,True,26.0,14.0,8.0,...,0.250000,0.000000,22.0,9.0,8.0,0.0,0.0,0.409091,0.000000,0.0
3,3,1950,Yugoslavia,4,1,Mexico,True,26.0,14.0,8.0,...,0.250000,0.000000,35.0,14.0,14.0,0.0,0.0,0.400000,0.000000,0.0
4,4,1950,Brazil,2,0,Yugoslavia,True,77.0,65.0,17.0,...,6.000000,0.294118,26.0,14.0,8.0,2.0,0.0,0.538462,0.250000,0.0
5,5,1950,Switzerland,2,1,Mexico,True,22.0,9.0,8.0,...,0.000000,0.000000,35.0,14.0,14.0,0.0,0.0,0.400000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528,761,2014,Argentina,3,2,Nigeria,True,54.0,36.0,14.0,...,2.857143,0.142857,14.0,5.0,5.0,0.0,0.0,0.357143,0.000000,0.0
1529,762,2014,Iran,1,3,Bosnia Herzegovina,False,9.0,1.0,4.0,...,0.000000,0.000000,3.0,1.0,1.0,0.0,0.0,0.333333,0.000000,0.0
1530,763,2014,Germany,1,0,United States,True,80.0,61.0,16.0,...,5.875000,0.187500,22.0,6.0,8.0,0.0,0.0,0.272727,0.000000,0.0
1531,764,2014,Ghana,1,2,Portugal,False,9.0,4.0,3.0,...,0.000000,0.000000,22.0,13.0,6.0,6.0,0.0,0.590909,1.000000,0.0


In [4]:
train, test = train_test_split(matches, test_size=0.2)

In [5]:
network = Sequential([
    Input((len(input_cols),)),
    Normalization(),
    Dense(10, activation='sigmoid'),
    Dense(10, activation='sigmoid'),
    Dense(1, activation='sigmoid'),
])

network.layers[0].adapt(train[input_cols].values)

network.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy',],
)


network.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 5)              │            11 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │            60 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 192 (772.00 B)

 Trainable params: 181 (724.00 B)

 Non-trainable params: 11 (48.00 B)

In [6]:
network.fit(
    train[input_cols], train[output_col],
    epochs=50,
    batch_size=128,
)

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5302 - loss: 0.6960  
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5199 - loss: 0.6941 
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5468 - loss: 0.6892 
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5559 - loss: 0.6876 
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6028 - loss: 0.6829 
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6179 - loss: 0.6813 
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6367 - loss: 0.6786 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6393 - loss: 0.6765 
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6489 - loss: 0.6737 
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6492 - loss: 0.6728 
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6742 - loss: 0.6691 
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - accuracy: 0.6974 - loss: 0.6650

In [7]:
train_predictions = network.predict(train[input_cols])

accuracy_score(train[output_col], train_predictions.round())

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


0.7420382165605095

In [8]:
test_predictions = network.predict(test[input_cols])

accuracy_score(test[output_col], test_predictions.round())

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


0.7542372881355932

In [9]:
def predict(year, team1, team2):
    case_inputs = utils.build_inputs_for_match(year, team1, team2, input_cols)
    result = network.predict(case_inputs)
    
    if result > 0.5:
        winner = team1
    else:
        winner = team2
        
    print(result[0][0], '→', winner)

In [10]:
predict(1950, 'Mexico', 'Brazil')  # real result: 4-0 wins Brazil

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
0.21827768 → Brazil


In [11]:
predict(1990, 'United Arab Emirates', 'Colombia')  # real result: 2-0 wins Colombia

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
0.33296677 → Colombia


In [12]:
predict(2002, 'South Africa', 'Spain')  # real result: 2-3 wins Spain

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
0.31196573 → Spain


In [13]:
predict(2010, 'Japan', 'Cameroon')  # real result: 1-0 wins Japan

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
0.5524343 → Japan


In [14]:
predict(2014, 'Argentina', 'Germany')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
0.33470786 → Germany
